In [ ]:
!pip install kfp kfp-tekton

In [1]:
import kfp
from kfp.components import create_component_from_func
from kfp_tekton.compiler import TektonCompiler
import kfp.dsl as dsl



In [11]:
def add():
  return


def create_comp(f):
  return create_component_from_func(
    f, output_component_file=f'{f.__name__}_component.yaml')


In [57]:
def query_geodn_discovery():
    return

def regrid():
    return

def join_spatial():
    return

t_join_spatial = create_comp(join_spatial)


def join_spatio_temporal():
    return

def mask_clouds():
    return

def normalize():
    return

def prithvi_finetune():
    return

def test():
    return

def deploy_model_mesh():
    return

def postgis_connector():
    return

t_postgis_connector = create_comp(postgis_connector)

def generate_annotations():
    return

t_generate_annotations = create_comp(generate_annotations)



a = create_comp(query_geodn_discovery)
b = create_comp(regrid)
c = create_comp(query_geodn_discovery)
d = create_comp(regrid)
e = create_comp(join_spatio_temporal)
f = create_comp(mask_clouds)
g = create_comp(normalize)
h = create_comp(prithvi_finetune)
i = create_comp(test)
j = create_comp(deploy_model_mesh)


def train_test_split():
    return

t_train_test_split = create_comp(train_test_split)



In [60]:
@dsl.pipeline(
  name='Addition pipeline',
  description='An example pipeline that performs addition calculations.'
)
def add_pipeline():
  bb = b()
  bb.after(a())
  dd = d()
  dd.after(c())
  ee = e()
  ee.after(bb, dd)

  f_postgis_connector = t_postgis_connector()

  f_generate_annotations = t_generate_annotations()

  f_generate_annotations.after(f_postgis_connector)


  f_join_spatial = t_join_spatial()

  f_join_spatial.after(ee,f_generate_annotations)


  gg = g().after(f().after(f_join_spatial))

  f_train_test_split = t_train_test_split()

  f_train_test_split.after(gg)

  hh = h()

  j().after(i().after(f_train_test_split, hh.after(f_train_test_split)))


# Create a pipeline run, using the client you initialized in a prior step.
TektonCompiler().compile(add_pipeline, 'climate.yaml')


In [ ]:
import kfp.components as comp
 
#volume_op = comp.load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/master/components/kubernetes/volume/component.yaml')
#input_url = comp.load_component_from_url('https://raw.githubusercontent.com/romeokienzler/component-library/main/component-library/input/input-url.yaml')
input_url = comp.load_component_from_file('/home/romeokienzler/gitco/component-library/component-library/input/input-url.yaml')
#upload_to_cos = comp.load_component_from_url('https://raw.githubusercontent.com/romeokienzler/component-library/main/component-library/output/upload-to-cos.yaml')
upload_to_cos = comp.load_component_from_file('/home/romeokienzler/gitco/component-library/component-library/output/upload-to-cos.yaml')

In [ ]:
import kfp.dsl as dsl
import kfp
from kfp_tekton.compiler import TektonCompiler

# Define your pipeline function
@dsl.pipeline(
    name="My Pipeline",
    description="A sample Kubeflow pipeline"
)
def my_pipeline():
    step1 = input_url(
            url='https://github.com/claimed-framework/component-library/raw/main/hmp.parquet',
            data_dir='.'
    )

    step2 = upload_to_cos(
        access_key_id='adbb855da5004670b679dcf5d0fc3028',
        secret_access_key='a7a94d0029fa2f44827a5b5da1ad0693c4ed87a843f6ea30',
        endpoint='https://s3.us-east.cloud-object-storage.appdomain.cloud',
        bucket_name='era5-cropscape-zarr',
        source_file=step1.output,
        destination_file='hmp.parquet',
        data_dir=''
    )


    pvc_mount_path = '/data'
    pvc_name = 'kfp-data-pvc'

    # Create a PipelineVolume to mount the PVC
    pipeline_volume = dsl.PipelineVolume(pvc_name)

    # Connect the steps with the mounted PVC
    step1.add_pvolumes({pvc_mount_path: pipeline_volume})
    step2.add_pvolumes({pvc_mount_path: pipeline_volume})
    
    step2.after(step1)

# Compile the pipeline
pipeline_func = my_pipeline
TektonCompiler().compile(pipeline_func, 'FirstPipeline.yaml')